(lightning_advanced_example)=

# Finetune a BERT Text Classifier with LightningTrainer

:::{note}

This is an advanced example for {class}`LightningTrainer <ray.train.lightning.LightningTrainer>`, which demonstrates how to use LightningTrainer with {ref}`Datastream <data>` and {ref}`Batch Predictor <air-predictors>`. 

If you just want to quickly convert your existing PyTorch Lightning scripts into Ray AIR, you can refer to this starter example:
{ref}`Train a Pytorch Lightning Image Classifier <lightning_mnist_example>`.

:::

In this demo, we will introduce how to finetune a text classifier on [CoLA(The Corpus of Linguistic Acceptability)](https://nyu-mll.github.io/CoLA/) datasets with pretrained BERT. 
In particular, we will:
- Create Ray Data from the original CoLA dataset.
- Define a preprocessor to tokenize the sentences.
- Finetune a BERT model using LightningTrainer.
- Construct a BatchPredictor with the checkpoint and preprocessor.
- Do batch prediction on multiple GPUs, and evaluate the results.

In [5]:
SMOKE_TEST = True

Run the following line in order to install all the necessary dependencies:

In [6]:
!pip install numpy datasets "transformers>=4.19.1" "pytorch_lightning>=1.6.5"

Let's start by importing the needed libraries:

In [ ]:
import ray
import torch
import pytorch_lightning as pl
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset, load_metric
import numpy as np

## Pre-process CoLA Datastream

CoLA is a binary sentence classification task with 10.6K training examples. First, we download the dataset and metrics using the HuggingFace API, and create Ray Data for each split accordingly.

In [ ]:
dataset = load_dataset("glue", "cola")
metric = load_metric("glue", "cola")

ray_datasets = ray.data.from_huggingface(dataset)

Next, define a preprocessor that tokenizes the input sentences and pads the ID sequence to length 128 using the bert-base-uncased tokenizer. The preprocessor transforms all datasets that we provide to the LightningTrainer later.

In [ ]:
from ray.data.preprocessors import BatchMapper

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_sentence(batch):
    encoded_sent = tokenizer(
        batch["sentence"].tolist(),
        max_length=128,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )
    batch["input_ids"] = encoded_sent["input_ids"].numpy()
    batch["attention_mask"] = encoded_sent["attention_mask"].numpy()
    batch["label"] = np.array(batch["label"])
    batch.pop("sentence")
    return batch


preprocessor = BatchMapper(tokenize_sentence, batch_format="numpy")

## Define a PyTorch Lightning Model

You don't have to make any change of your `LightningModule` definition. Just copy and paste your code here:

In [11]:
class SentimentModel(pl.LightningModule):
    def __init__(self, lr=2e-5, eps=1e-8):
        super().__init__()
        self.lr = lr
        self.eps = eps
        self.num_classes = 2
        self.model = AutoModelForSequenceClassification.from_pretrained(
            "bert-base-cased", num_labels=self.num_classes
        )
        self.metric = load_metric("glue", "cola")
        self.predictions = []
        self.references = []

    def forward(self, batch):
        input_ids, attention_mask = batch["input_ids"], batch["attention_mask"]
        outputs = self.model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        return logits

    def training_step(self, batch, batch_idx):
        labels = batch["label"]
        logits = self.forward(batch)
        loss = F.cross_entropy(logits.view(-1, self.num_classes), labels)
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        labels = batch["label"]
        logits = self.forward(batch)
        preds = torch.argmax(logits, dim=1)
        self.predictions.append(preds)
        self.references.append(labels)

    def on_validation_epoch_end(self):
        predictions = torch.concat(self.predictions).view(-1)
        references = torch.concat(self.references).view(-1)
        matthews_correlation = self.metric.compute(
            predictions=predictions, references=references
        )

        # self.metric.compute() returns a dictionary:
        # e.g. {"matthews_correlation": 0.53}
        self.log_dict(matthews_correlation, sync_dist=True)
        self.predictions.clear()
        self.references.clear()

    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=self.lr, eps=self.eps)

## Configure your LightningTrainer

Define a LightningTrainer with necessary configurations, including hyper-parameters, checkpointing and compute resources settings. 

You may find the API of {class}`LightningConfigBuilder <ray.train.lightning.LightningConfigBuilder>` and the discussion {ref}`here <lightning-config-builder-intro>` useful.



In [12]:
from ray.train.lightning import LightningTrainer, LightningConfigBuilder
from ray.air.config import RunConfig, ScalingConfig, CheckpointConfig

# Define the configs for LightningTrainer
lightning_config = (
    LightningConfigBuilder()
    .module(cls=SentimentModel, lr=1e-5, eps=1e-8)
    .trainer(max_epochs=5, accelerator="gpu")
    .checkpointing(save_on_train_epoch_end=False)
    .build()
)

:::{note}
Note that the `lightning_config` is created on the head node and will be passed to the worker nodes later. Be aware that the environment variables and hardware settings may differ between the head node and worker nodes.
:::

:::{note}
{meth}`LightningConfigBuilder.checkpointing() <ray.train.lightning.LightningConfigBuilder.checkpointing>` creates a [ModelCheckpoint](https://lightning.ai/docs/pytorch/stable/api/lightning.pytorch.callbacks.ModelCheckpoint.html#lightning.pytorch.callbacks.ModelCheckpoint) callback. This callback defines the checkpoint frequency and saves checkpoint files in Lightning style. 

If you want to save AIR checkpoints for Batch Prediction, please also provide an AIR {class}`CheckpointConfig <ray.air.config.CheckpointConfig>`.
:::

In [13]:
# Save AIR checkpoints according to the performance on validation set
run_config = RunConfig(
    name="ptl-sent-classification",
    checkpoint_config=CheckpointConfig(
        num_to_keep=2,
        checkpoint_score_attribute="matthews_correlation",
        checkpoint_score_order="max",
    ),
)

# Scale the DDP training workload across 4 GPUs
# You can change this config based on your compute resources.
scaling_config = ScalingConfig(
    num_workers=4, use_gpu=True, resources_per_worker={"CPU": 1, "GPU": 1}
)

In [14]:
if SMOKE_TEST:
    lightning_config = (
        LightningConfigBuilder()
        .module(cls=SentimentModel, lr=1e-5, eps=1e-8)
        .trainer(max_epochs=2, accelerator="gpu")
        .checkpointing(save_on_train_epoch_end=False)
        .build()
    )

    for split, ds in ray_datasets.items():
        ray_datasets[split] = ds.random_sample(0.1)

## Fine-tune the model with LightningTrainer

Train the model with the configuration we specified above. 

To feed data into LightningTrainer, we need to configure the following arguments:

- `datasets`: A dictionary of the input Ray datastreams, with special keys "train" and "val".
- `datasets_iter_config`: The argument list of {meth}`iter_torch_batches() <ray.data.Datastream.iter_torch_batches>`. It defines the way we iterate dataset shards for each worker.
- `preprocessor`: The preprocessor that will be applied to the input dataset.

:::{note}
Note that we are using Datastream for data ingestion for faster preprocessing here, but you can also continue to use the native `PyTorch DataLoader` or `LightningDataModule`. See {ref}`this example <lightning_mnist_example>`. 

:::


Now, call `trainer.fit()` to initiate the training process.

In [15]:
trainer = LightningTrainer(
    lightning_config=lightning_config,
    run_config=run_config,
    scaling_config=scaling_config,
    datasets={"train": ray_datasets["train"], "val": ray_datasets["validation"]},
    datasets_iter_config={"batch_size": 16},
    preprocessor=preprocessor,
)
result = trainer.fit()

(pid=67819) /home/ray/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(pid=67819)   from pandas import MultiIndex, Int64Index
(LightningTrainer pid=67819) 2023-04-24 10:36:31,679	INFO backend_executor.py:128 -- Starting distributed worker processes: ['68396 (10.0.60.127)', '68397 (10.0.60.127)', '68398 (10.0.60.127)', '68399 (10.0.60.127)']
(RayTrainWorker pid=68396) 2023-04-24 10:36:32,731	INFO config.py:86 -- Setting up process group for: env:// [rank=0, world_size=4]


(pid=67819) - RandomizeBlockOrder 1:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=67819) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(LightningTrainer pid=67819) 2023-04-24 10:36:34,052	INFO streaming_executor.py:87 -- Executing DAG InputDataBuffer[Input] -> TaskPoolMapOperator[BatchMapper] -> AllToAllOperator[RandomizeBlockOrder]
(LightningTrainer pid=67819) 2023-04-24 10:36:34,052	INFO streaming_executor.py:88 -- Execution config: ExecutionOptions(resource_limits=ExecutionResources(cpu=None, gpu=None, object_store_memory=None), locality_with_output=False, preserve_order=False, actor_locality_enabled=True, verbose_progress=False)
(LightningTrainer pid=67819) 2023-04-24 10:36:34,053	INFO streaming_executor.py:90 -- Tip: To enable per-operator progress reporting, set RAY_DATA_VERBOSE_PROGRESS=1.
(RayTrainWorker pid=68396) /home/ray/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
(RayTrainWorker pid=68396)   from pandas import MultiIndex, Int64Index
Downlo

(pid=68398) - RandomizeBlockOrder 1:   0%|          | 0/1 [00:00<?, ?it/s]

...

(pid=68397) - RandomizeBlockOrder 1:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=68397) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=68399) - RandomizeBlockOrder 1:   0%|          | 0/1 [00:00<?, ?it/s]

(pid=68399) Running 0:   0%|          | 0/1 [00:00<?, ?it/s]

2023-04-24 10:42:50,016	INFO tune.py:1010 -- Total run time: 387.00 seconds (386.94 seconds for the tuning loop).


:::{note}
Note that we are using Ray Data for data ingestion for faster preprocessing here, but you can also continue to use the native `PyTorch DataLoader` or `LightningDataModule`. See {ref}`this example <lightning_mnist_example>`. 

:::

In [16]:
result

Result(
  metrics={'_report_on': 'validation_end', 'train_loss': 0.011980690062046051, 'matthews_correlation': 0.5899314497879129, 'epoch': 4, 'step': 670, 'should_checkpoint': True, 'done': True, 'trial_id': '87ecf_00000', 'experiment_tag': '0'},
  path='/home/ray/ray_results/ptl-sent-classification/LightningTrainer_87ecf_00000_0_2023-04-24_10-36-23',
  checkpoint=LightningCheckpoint(local_path=/home/ray/ray_results/ptl-sent-classification/LightningTrainer_87ecf_00000_0_2023-04-24_10-36-23/checkpoint_000004)
)

## Do Batch Inference with a Saved Checkpoint

Now that we have fine-tuned the module, we can load the checkpoint into a BatchPredictor and perform fast inference with multiple GPUs. It will distribute the inference workload across multiple workers when calling `predict()` and run prediction on multiple shards of data in parallel. 

You can find more details in [Using Predictors for Inference](air-predictors).

In [17]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.lightning import LightningCheckpoint, LightningPredictor

# Use in-memory checkpoint object
checkpoint = result.checkpoint

# You can also load a checkpoint from disk:
# YOUR_CHECKPOINT_DIR = result.checkpoint.path
# checkpoint = LightningCheckpoint.from_directory(YOUR_CHECKPOINT_DIR)

batch_predictor = BatchPredictor(
    checkpoint=checkpoint,
    predictor_cls=LightningPredictor,
    use_gpu=True,
    model_class=SentimentModel,
    preprocessor=preprocessor,
)

# Use 2 GPUs for batch inference
predictions = batch_predictor.predict(
    ray_datasets["validation"],
    feature_columns=["input_ids", "attention_mask", "label"],
    keep_columns=["label"],
    batch_size=16,
    min_scoring_workers=2,
    max_scoring_workers=2,
    num_gpus_per_worker=1,
)

We obtained a Ray datastream containing predictions from `batch_predictor.predict()`. Now we can easily evaluate the results with just a few lines of code:

In [ ]:
# Internally, BatchPredictor calls forward() method of the LightningModule.
# Convert the logits tensor into labels with argmax.
def argmax(batch):
    batch["predictions"] = batch["predictions"].apply(lambda x: np.argmax(x))
    return batch


results = predictions.map_batches(argmax, batch_format="pandas").to_pandas()

In [23]:
matthews_corr = metric.compute(
    predictions=results["predictions"], references=results["label"]
)
print(results.head(10))
print(matthews_corr)

   predictions  label
0            1      1
1            1      1
2            0      1
3            1      1
4            0      0
5            1      0
6            1      0
7            1      1
8            1      1
9            1      1

{'matthews_correlation': 0.5899314497879129}
